#### Hensman 2013 (http://www.auai.org/uai2013/prints/papers/244.pdf) they chose to include into our model 8 of the many variables available for this dataset: the age of the aircraft (number of years since deployment), distance that needs to be covered, airtime, departure time, arrival time, day of the week, day of the month and month

#### This notebook will duplicate the appropriate priedictors for comparison.

In [21]:
import bz2
import time
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
#import predictors as p

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import os
print(os.getcwd())

/Users/omarchavez/UT/communal_monte_carlo


In [22]:
import math
def get_hour(x):
    return int(math.floor(x / 100.0))

def get_items_greater_than_cut(series, cut=50):
    counts = series.value_counts()
    index = counts.index.values[counts >= cut]
    return(list(index))

def get_dep_arr_time_buckets(X):
    X_CRS_DEP_ARR_TIME = pd.DataFrame(
        {
            'CRS_DEP_TIME_BLK_0600-0659':((X.CRS_DEP_TIME >=  600) & (X.CRS_DEP_TIME <=  659)).apply(int),
            'CRS_DEP_TIME_BLK_0700-0759':((X.CRS_DEP_TIME >=  700) & (X.CRS_DEP_TIME <=  759)).apply(int),
            'CRS_DEP_TIME_BLK_0800-0859':((X.CRS_DEP_TIME >=  800) & (X.CRS_DEP_TIME <=  859)).apply(int),
            'CRS_DEP_TIME_BLK_0900-0959':((X.CRS_DEP_TIME >=  900) & (X.CRS_DEP_TIME <=  959)).apply(int),
            'CRS_DEP_TIME_BLK_1000-1059':((X.CRS_DEP_TIME >= 1000) & (X.CRS_DEP_TIME <= 1059)).apply(int),
            'CRS_DEP_TIME_BLK_1100-1159':((X.CRS_DEP_TIME >= 1100) & (X.CRS_DEP_TIME <= 1159)).apply(int),
            'CRS_DEP_TIME_BLK_1200-1259':((X.CRS_DEP_TIME >= 1200) & (X.CRS_DEP_TIME <= 1259)).apply(int),
            'CRS_DEP_TIME_BLK_1300-1359':((X.CRS_DEP_TIME >= 1300) & (X.CRS_DEP_TIME <= 1359)).apply(int),
            'CRS_DEP_TIME_BLK_1400-1459':((X.CRS_DEP_TIME >= 1400) & (X.CRS_DEP_TIME <= 1459)).apply(int),
            'CRS_DEP_TIME_BLK_1500-1559':((X.CRS_DEP_TIME >= 1500) & (X.CRS_DEP_TIME <= 1559)).apply(int),
            'CRS_DEP_TIME_BLK_1600-1659':((X.CRS_DEP_TIME >= 1600) & (X.CRS_DEP_TIME <= 1659)).apply(int),
            'CRS_DEP_TIME_BLK_1700-1759':((X.CRS_DEP_TIME >= 1700) & (X.CRS_DEP_TIME <= 1759)).apply(int),
            'CRS_DEP_TIME_BLK_1800-1859':((X.CRS_DEP_TIME >= 1800) & (X.CRS_DEP_TIME <= 1859)).apply(int),
            'CRS_DEP_TIME_BLK_1900-1959':((X.CRS_DEP_TIME >= 1900) & (X.CRS_DEP_TIME <= 1959)).apply(int),
            'CRS_DEP_TIME_BLK_2000-2059':((X.CRS_DEP_TIME >= 2000) & (X.CRS_DEP_TIME <= 2059)).apply(int),
            'CRS_DEP_TIME_BLK_2100-2159':((X.CRS_DEP_TIME >= 2100) & (X.CRS_DEP_TIME <= 2159)).apply(int),
            'CRS_DEP_TIME_BLK_2200-2259':((X.CRS_DEP_TIME >= 2200) & (X.CRS_DEP_TIME <= 2259)).apply(int),
            'CRS_DEP_TIME_BLK_2300-2359':((X.CRS_DEP_TIME >= 2300) & (X.CRS_DEP_TIME <= 2359)).apply(int),
            
            'CRS_ARR_TIME_BLK_0600-0659':((X.CRS_ARR_TIME >=  600) & (X.CRS_ARR_TIME <=  659)).apply(int),
            'CRS_ARR_TIME_BLK_0700-0759':((X.CRS_ARR_TIME >=  700) & (X.CRS_ARR_TIME <=  759)).apply(int),
            'CRS_ARR_TIME_BLK_0800-0859':((X.CRS_ARR_TIME >=  800) & (X.CRS_ARR_TIME <=  859)).apply(int),
            'CRS_ARR_TIME_BLK_0900-0959':((X.CRS_ARR_TIME >=  900) & (X.CRS_ARR_TIME <=  959)).apply(int),
            'CRS_ARR_TIME_BLK_1000-1059':((X.CRS_ARR_TIME >= 1000) & (X.CRS_ARR_TIME <= 1059)).apply(int),
            'CRS_ARR_TIME_BLK_1100-1159':((X.CRS_ARR_TIME >= 1100) & (X.CRS_ARR_TIME <= 1159)).apply(int),
            'CRS_ARR_TIME_BLK_1200-1259':((X.CRS_ARR_TIME >= 1200) & (X.CRS_ARR_TIME <= 1259)).apply(int),
            'CRS_ARR_TIME_BLK_1300-1359':((X.CRS_ARR_TIME >= 1300) & (X.CRS_ARR_TIME <= 1359)).apply(int),
            'CRS_ARR_TIME_BLK_1400-1459':((X.CRS_ARR_TIME >= 1400) & (X.CRS_ARR_TIME <= 1459)).apply(int),
            'CRS_ARR_TIME_BLK_1500-1559':((X.CRS_ARR_TIME >= 1500) & (X.CRS_ARR_TIME <= 1559)).apply(int),
            'CRS_ARR_TIME_BLK_1600-1659':((X.CRS_ARR_TIME >= 1600) & (X.CRS_ARR_TIME <= 1659)).apply(int),
            'CRS_ARR_TIME_BLK_1700-1759':((X.CRS_ARR_TIME >= 1700) & (X.CRS_ARR_TIME <= 1759)).apply(int),
            'CRS_ARR_TIME_BLK_1800-1859':((X.CRS_ARR_TIME >= 1800) & (X.CRS_ARR_TIME <= 1859)).apply(int),
            'CRS_ARR_TIME_BLK_1900-1959':((X.CRS_ARR_TIME >= 1900) & (X.CRS_ARR_TIME <= 1959)).apply(int),
            'CRS_ARR_TIME_BLK_2000-2059':((X.CRS_ARR_TIME >= 2000) & (X.CRS_ARR_TIME <= 2059)).apply(int),
            'CRS_ARR_TIME_BLK_2100-2159':((X.CRS_ARR_TIME >= 2100) & (X.CRS_ARR_TIME <= 2159)).apply(int),
            'CRS_ARR_TIME_BLK_2200-2259':((X.CRS_ARR_TIME >= 2200) & (X.CRS_ARR_TIME <= 2259)).apply(int),
            'CRS_ARR_TIME_BLK_2300-2359':((X.CRS_ARR_TIME >= 2300) & (X.CRS_ARR_TIME <= 2359)).apply(int),
        }
    )
    return X_CRS_DEP_ARR_TIME

ARR_DEP_COLUMNS = [
    'CRS_DEP_TIME_BLK_0600-0659',
    'CRS_DEP_TIME_BLK_0700-0759',
    'CRS_DEP_TIME_BLK_0800-0859',
    'CRS_DEP_TIME_BLK_0900-0959',
    'CRS_DEP_TIME_BLK_1000-1059',
    'CRS_DEP_TIME_BLK_1100-1159',
    'CRS_DEP_TIME_BLK_1200-1259',
    'CRS_DEP_TIME_BLK_1300-1359',
    'CRS_DEP_TIME_BLK_1400-1459',
    'CRS_DEP_TIME_BLK_1500-1559',
    'CRS_DEP_TIME_BLK_1600-1659',
    'CRS_DEP_TIME_BLK_1700-1759',
    'CRS_DEP_TIME_BLK_1800-1859',
    'CRS_DEP_TIME_BLK_1900-1959',
    'CRS_DEP_TIME_BLK_2000-2059',
    'CRS_DEP_TIME_BLK_2100-2159',
    'CRS_DEP_TIME_BLK_2200-2259',
    'CRS_DEP_TIME_BLK_2300-2359',
    
    'CRS_ARR_TIME_BLK_0600-0659',
    'CRS_ARR_TIME_BLK_0700-0759',
    'CRS_ARR_TIME_BLK_0800-0859',
    'CRS_ARR_TIME_BLK_0900-0959',
    'CRS_ARR_TIME_BLK_1000-1059',
    'CRS_ARR_TIME_BLK_1100-1159',
    'CRS_ARR_TIME_BLK_1200-1259',
    'CRS_ARR_TIME_BLK_1300-1359',
    'CRS_ARR_TIME_BLK_1400-1459',
    'CRS_ARR_TIME_BLK_1500-1559',
    'CRS_ARR_TIME_BLK_1600-1659',
    'CRS_ARR_TIME_BLK_1700-1759',
    'CRS_ARR_TIME_BLK_1800-1859',
    'CRS_ARR_TIME_BLK_1900-1959',
    'CRS_ARR_TIME_BLK_2000-2059',
    'CRS_ARR_TIME_BLK_2100-2159',
    'CRS_ARR_TIME_BLK_2200-2259',
    'CRS_ARR_TIME_BLK_2300-2359',
]

def get_dow_dom_buckets(X):
    X_DOW_DOM = pd.DataFrame(
        {
            #'DOW_1':((X.DAY_OF_WEEK ==  1) ).apply(int),
            'DOW_2':((X.DAY_OF_WEEK ==  2) ).apply(int),
            'DOW_3':((X.DAY_OF_WEEK ==  3) ).apply(int),
            'DOW_4':((X.DAY_OF_WEEK ==  4) ).apply(int),
            'DOW_5':((X.DAY_OF_WEEK ==  5) ).apply(int),
            'DOW_6':((X.DAY_OF_WEEK ==  6) ).apply(int),
            'DOW_7':((X.DAY_OF_WEEK ==  7) ).apply(int),
        
            #'DOW_1':((X.DAY_OF_MONTH ==  1) ).apply(int),
            'DOM_2':((X.DAY_OF_MONTH ==  2) ).apply(int),
            'DOM_3':((X.DAY_OF_MONTH ==  3) ).apply(int),
            'DOM_4':((X.DAY_OF_MONTH ==  4) ).apply(int),
            'DOM_5':((X.DAY_OF_MONTH ==  5) ).apply(int),
            'DOM_6':((X.DAY_OF_MONTH ==  6) ).apply(int),
            'DOM_7':((X.DAY_OF_MONTH ==  7) ).apply(int),
            'DOM_8':((X.DAY_OF_MONTH ==  8) ).apply(int),
            'DOM_9':((X.DAY_OF_MONTH ==  9) ).apply(int),
            'DOM_10':((X.DAY_OF_MONTH ==  10) ).apply(int),
            'DOM_11':((X.DAY_OF_MONTH ==  11) ).apply(int),
            'DOM_12':((X.DAY_OF_MONTH ==  12) ).apply(int),
            'DOM_13':((X.DAY_OF_MONTH ==  13) ).apply(int),
            'DOM_14':((X.DAY_OF_MONTH ==  14) ).apply(int),
            'DOM_15':((X.DAY_OF_MONTH ==  15) ).apply(int),
            'DOM_16':((X.DAY_OF_MONTH ==  16) ).apply(int),
            'DOM_17':((X.DAY_OF_MONTH ==  17) ).apply(int),
            'DOM_18':((X.DAY_OF_MONTH ==  18) ).apply(int),
            'DOM_19':((X.DAY_OF_MONTH ==  19) ).apply(int),
            'DOM_20':((X.DAY_OF_MONTH ==  20) ).apply(int),
            'DOM_21':((X.DAY_OF_MONTH ==  21) ).apply(int),
            'DOM_22':((X.DAY_OF_MONTH ==  22) ).apply(int),
            'DOM_23':((X.DAY_OF_MONTH ==  23) ).apply(int),
            'DOM_24':((X.DAY_OF_MONTH ==  24) ).apply(int),
            'DOM_25':((X.DAY_OF_MONTH ==  25) ).apply(int),
            'DOM_26':((X.DAY_OF_MONTH ==  26) ).apply(int),
            'DOM_27':((X.DAY_OF_MONTH ==  27) ).apply(int),
            'DOM_28':((X.DAY_OF_MONTH ==  28) ).apply(int),
            'DOM_29':((X.DAY_OF_MONTH ==  29) ).apply(int),
            'DOM_30':((X.DAY_OF_MONTH ==  30) ).apply(int),
            'DOM_31':((X.DAY_OF_MONTH ==  31) ).apply(int),
            
        }
    )
    return X_DOW_DOM

DOW_DOM_COLUMNS = [
    'DOW_2',
    'DOW_3',
    'DOW_4',
    'DOW_5',
    'DOW_6',
    'DOW_7',

    'DOM_2',
    'DOM_3',
    'DOM_4',
    'DOM_5',
    'DOM_6',
    'DOM_7',
    'DOM_8',
    'DOM_9',
    'DOM_10',
    'DOM_11',
    'DOM_12',
    'DOM_13',
    'DOM_14',
    'DOM_15',
    'DOM_16',
    'DOM_17',
    'DOM_18',
    'DOM_19',
    'DOM_20',
    'DOM_21',
    'DOM_22',
    'DOM_23',
    'DOM_24',
    'DOM_25',
    'DOM_26',
    'DOM_27',
    'DOM_28',
    'DOM_29',
    'DOM_30',
    'DOM_31',
]

Tau_inv_std = 0.02 #0.21863912984342926 

Bo_std = 0.8127597440769186

#### time since start time in minutesget all cariers, origins and destinations

In [23]:
year_month_files = [
     '2012_November'
    ,'2012_December'
    ,'2013_January'
    ,'2013_February'
    ,'2013_March'
    ,'2013_April'
    ,'2013_May'
    ,'2013_June'
    ,'2013_July'
    ,'2013_August'
    ,'2013_September'
    ,'2013_October'
]

In [24]:
WRITE_COLUMNS = (
    ['intercept','aircraft_age','DISTANCE','CRS_ELAPSED_TIME'] + 
     DOW_DOM_COLUMNS +
     ARR_DEP_COLUMNS + 
    ["y", "time"] #, "Tau_inv_std", "Bo_std", "time"]
)

PREDICTORS = (
    ['intercept','aircraft_age','DISTANCE','CRS_ELAPSED_TIME'] + 
    ARR_DEP_COLUMNS #+ DOW_DOM_COLUMNS
)

In [43]:

"""
DONE - the age of the aircraft (number of years since deployment), 
DONE - distance that needs to be covered, 
DONE - airtime, 
DONE - departure time, 
DONE - arrival time, 
DONE - day of the week, 
DONE - day of the month and month

"""


#total_cat_count = len(unique_CARRIER) + len(unique_ORIGIN) + len(unique_DEST)

plane_data_path = 'data/hold/' + 'plane-data' + '.csv'
plane_df = pd.read_csv(plane_data_path)

for ymi in range(len(year_month_files)):

    path = 'data/hold/' + year_month_files[ymi] + '.csv'
    print(path)
    df = pd.read_csv(path)
    # drop rows of cancelled flights
    has_time = ~df.DEP_TIME.isna()
    df = df.loc[has_time, :]
    # drop rows of no event record
    has_event = ~df.ARR_DEL15.isna()
    df = df.loc[has_event, :]    
    # join tail number data
    df      = df.merge(plane_df, left_on='TAIL_NUM', right_on='tailnum')
    df      = df[df.year != 'None']
    df      = df[~df.year.isna()]
    df.year = df.year.astype(int)
    
    # get aircraft age
    df.loc[:,'aircraft_age'] = df.YEAR - df.year
    
    # add minutes since start of recorded time    
    
    df = df.sort_values(by=['YEAR','MONTH','DAY_OF_MONTH','DEP_TIME'])
    df = df.reset_index(drop=True)
    #print("shape: ", df.shape)
    
    start_time = datetime.datetime(2012, 11, 1)
    date_df = df[['YEAR','MONTH','DAY_OF_MONTH']]
    date_df.columns = ['YEAR','MONTH','DAY']
    
    date_df.loc[:,'HOUR'] = df.DEP_TIME.apply(get_hour)
    date_df.loc[:,'MINUTE'] = df.DEP_TIME % 60

    df['datetime_DEP_TIME'] = pd.to_datetime(date_df[['YEAR','MONTH','DAY', 'HOUR', 'MINUTE']])
    
    arr_dep_time_dummies = get_dep_arr_time_buckets(X = df).reset_index(drop=True)
    
    dow_dom_dummies = get_dow_dom_buckets(X = df).reset_index(drop=True)

    # intercept
    INT = pd.DataFrame({'intercept':np.ones(df.shape[0])})
    
    # compile results
    write_df = pd.concat(
        [
            df, 
            arr_dep_time_dummies, 
            dow_dom_dummies,
            INT], 
        axis = 1
    )
    #print("write_df: ", write_df.shape)
    write_df.insert(write_df.shape[1], 'y', df.ARR_DEL15, allow_duplicates = False)
    #write_df['y'] = df.ARR_DEL15
    #write_df.insert(write_df.shape[1], 'Tau_inv_std', Tau_inv_std, allow_duplicates = False)
    #write_df['Tau_inv_std'] = Tau_inv_std
    #write_df.insert(write_df.shape[1], 'Bo_std', Bo_std, allow_duplicates = False)
    #write_df['Bo_std'] = Bo_std
    time_insert =  np.floor(
        pd.to_timedelta(df['datetime_DEP_TIME'] - start_time
    ).dt.total_seconds()/(60)).astype(int)
    write_df.insert(write_df.shape[1], 'time', time_insert, allow_duplicates = False)
    write_df = write_df[WRITE_COLUMNS]
    #print(write_df['time'])
    #break
    write_df = write_df.sort_values(by=['time'])
    write_df = write_df.dropna(axis='columns')
    
    days_of_month = df.DAY_OF_MONTH.unique()
    days_of_month = np.sort(days_of_month)
    for domi in range(len(days_of_month)):
        #print("DAY = ", days_of_month[domi])
        dom_write_df = write_df.loc[df.DAY_OF_MONTH == days_of_month[domi],:]
        dom_write_df = dom_write_df.reset_index(drop=True)
        
        #print("dom_write_df: ", dom_write_df.shape)

        if dom_write_df.isnull().values.any():
            print("WARNING - MISSING VALUES IN DAY", days_of_month[domi])
                    
        out_path = 'data/hensman_t_in_min/HENSMAN_X_' + year_month_files[ymi] + "_" + str(days_of_month[domi])        
        dom_write_df.to_csv(out_path + '.csv', index=False, header=False)
        #dom_write_df.to_hdf(out_path + '.h5', key='dom_df', mode='w')

data/hold/2012_November.csv
data/hold/2012_December.csv
data/hold/2013_January.csv
data/hold/2013_February.csv
data/hold/2013_March.csv
data/hold/2013_April.csv
data/hold/2013_May.csv
data/hold/2013_June.csv
data/hold/2013_July.csv
data/hold/2013_August.csv
data/hold/2013_September.csv
data/hold/2013_October.csv


In [27]:
#df_with_tailnum.YEAR - df_with_tailnum.year
dom_write_df.head()

,intercept,aircraft_age,DISTANCE,CRS_ELAPSED_TIME,DOW_2,DOW_3,DOW_4,DOW_5,DOW_6,DOW_7,DOM_2,DOM_3,DOM_4,DOM_5,DOM_6,DOM_7,DOM_8,DOM_9,DOM_10,DOM_11,DOM_12,DOM_13,DOM_14,DOM_15,DOM_16,DOM_17,DOM_18,DOM_19,DOM_20,DOM_21,DOM_22,DOM_23,DOM_24,DOM_25,DOM_26,DOM_27,DOM_28,DOM_29,DOM_30,DOM_31,CRS_DEP_TIME_BLK_0600-0659,CRS_DEP_TIME_BLK_0700-0759,CRS_DEP_TIME_BLK_0800-0859,CRS_DEP_TIME_BLK_0900-0959,CRS_DEP_TIME_BLK_1000-1059,CRS_DEP_TIME_BLK_1100-1159,CRS_DEP_TIME_BLK_1200-1259,CRS_DEP_TIME_BLK_1300-1359,CRS_DEP_TIME_BLK_1400-1459,CRS_DEP_TIME_BLK_1500-1559,CRS_DEP_TIME_BLK_1600-1659,CRS_DEP_TIME_BLK_1700-1759,CRS_DEP_TIME_BLK_1800-1859,CRS_DEP_TIME_BLK_1900-1959,CRS_DEP_TIME_BLK_2000-2059,CRS_DEP_TIME_BLK_2100-2159,CRS_DEP_TIME_BLK_2200-2259,CRS_DEP_TIME_BLK_2300-2359,CRS_ARR_TIME_BLK_0600-0659,CRS_ARR_TIME_BLK_0700-0759,CRS_ARR_TIME_BLK_0800-0859,CRS_ARR_TIME_BLK_0900-0959,CRS_ARR_TIME_BLK_1000-1059,CRS_ARR_TIME_BLK_1100-1159,CRS_ARR_TIME_BLK_1200-1259,CRS_ARR_TIME_BLK_1300-1359,CRS_ARR_TIME_BLK_1400-1459,CRS_ARR_TIME_BLK_1500-1559,CRS_ARR_TIME_BLK_1600-1659,CRS_ARR_TIME_BLK_1700-1759,CRS_ARR_TIME_BLK_1800-1859,CRS_ARR_TIME_BLK_1900-1959,CRS_ARR_TIME_BLK_2000-2059,CRS_ARR_TIME_BLK_2100-2159,CRS_ARR_TIME_BLK_2200-2259,CRS_ARR_TIME_BLK_2300-2359,y,time
0,1.0,7,2139.0,270.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,524172
1,1.0,12,1635.0,216.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,524172
2,1.0,15,1846.0,235.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1.0,524182
3,1.0,12,2125.0,276.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,524185
4,1.0,11,1399.0,187.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,524187


In [ ]:
#save parameter values for each month for each parameter in matrix of shape
# Betas count X Months to get a feel for Tau_inv_std
all_params = np.zeros((12*31,len(PREDICTORS)))
params_flat = []
counter = 0
for fn in range(len(year_month_files)):
    
    for domi in range(1,32):
        path = 'data/HENSMAN_X_'+ year_month_files[fn]+'_'+str(domi)+ '.csv'
        
        if os.path.isfile(path):
            print(path)
            try:
                first_df = pd.read_csv(path)
                good_df = first_df
                X = good_df.loc[:, PREDICTORS]
                y = good_df.loc[:,['y']]
                
                logit_model=sm.Logit(y,X)
                result=logit_model.fit()
                all_params[counter, :] = result.params.values
                counter += 1
                params_flat = np.concatenate([params_flat, result.params.values])
                if (fn == 0) & (domi==1):
                    all_diffs = params_flat
                    last_set = result
                else:
                    all_diffs = np.concatenate([all_diffs, result.params.values - last_set.params.values])
                    last_set = result
            except Exception:
                print("****************************************************")
                print("****************** BAD DAY TO FLY ******************")
                print(path)
                print("****************************************************")
        else:
            print("FAIL: ",path)

In [ ]:
#result.params.values
X.head()

In [ ]:
plt.figure(figsize=(20,15))

plt.ylim((-5,5))
for i in range(all_params.shape[1]):
    plt.plot(all_params[:, i])

Tau_inv_std = pd.Series(np.abs(all_diffs)).quantile(0.3) #pd.Series(np.abs(all_diffs/43800)).quantile(0.99)#np.max(np.abs(all_diffs/43800))
Bo_std = np.std(all_params.flatten())

In [ ]:
Tau_inv_std


In [ ]:
Bo_std

In [ ]:
plt.plot(first_df.time)

In [ ]:
sums = X.sum(axis=0)
for i in range(len(sums)):
    for j in range(i+1, len(sums)):
        if sums.iloc[i] == sums.iloc[j]:
            print(sums.index.values[i], sums.iloc[i])
            print(sums.index.values[j], sums.iloc[j])
            print(X.loc[:, sums.index.values[i]].equals(X.loc[:, sums.index.values[j]]))
            print("###########################")

In [ ]:
#index = np.abs(all_diffs)/43800 < 0.0001
#_=plt.hist(all_diffs[index]/43800, 100)
_=plt.hist(all_diffs, 1000)
plt.xlim((-10,10))

In [ ]:
index = np.abs(all_diffs) < 3
_=plt.hist(np.abs(all_diffs[index]), 100)

In [ ]:
print(np.max(np.abs(all_diffs)))
print(np.mean(np.abs(all_diffs[index])))

In [ ]:
pd.Series(np.abs(all_diffs)).quantile(0.3)

In [ ]:
flat_params = all_params.flatten()
_=plt.hist(flat_params[np.abs(flat_params)<3], 100)

In [ ]:
24*60*4.956843223507737e-05

In [ ]:
year_month_files = [
     '2012_November'
    ,'2012_December'
    ,'2013_January'
    ,'2013_February'
    ,'2013_March'
    ,'2013_April'
    ,'2013_May'
    ,'2013_June'
    ,'2013_July'
    ,'2013_August'
    ,'2013_September'
    ,'2013_October'
]
start_time = time.time()

for ymf in range(len(year_month_files)):
    for i in range(1,32):
        try:
            h5_test = pd.read_hdf('data/HENSMAN_X_'+year_month_files[ymf]+'_'+str(i)+'.h5')
        except:
            print('FAIL: data/HENSMAN_X_' + year_month_files[ymf] + '_' + str(i)+'.h5')

end_time = time.time()
print("run time:", end_time-start_time)

In [ ]:
h5_test[1::10]

In [30]:
row_count = 334567
row_count - row_count%100


334500

In [42]:
timestepsize=-1
num_particles=3
np.random.normal(np.zeros(num_particles), timestepsize)

ValueError: scale < 0

In [41]:
sum([False, True])

1